In [4]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [6]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import sys

try:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='NIAID-SageMakeFullAccess')['Role']['Arn']
except ValueError:
	sys.exit()
	

# Hub Model configuration. https://huggingface.co/models
hub = {
    'HUGGING_FACE_HUB_TOKEN': os.getenv('HUGGING_FACE_HUB_TOKEN'),
    'HF_MODEL_ID':'meta-llama/Meta-Llama-3.1-8B-Instruct',
	'SM_NUM_GPUS': json.dumps(1)
}


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="2.2.0"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )
  


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/shresthap/Library/Application Support/sagemaker/config.yaml
----------------------*

UnexpectedStatusException: Error hosting endpoint huggingface-pytorch-tgi-inference-2024-08-12-18-34-19-873: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [2]:
import requests
from collections import deque
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pathlib
import os
import string

HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
root = '/'
nodes_list = deque() # stack
visited = set()      # set
nodes_list.append(root)
visited.add(root)
corpus = []
files_extension_to_exclude = ['.pdf', '.doc', '.docx', '.xls', '.xlsx',
                              'jpg', '.jpeg', '.png', '.gif', '.ico',
                              '.svg', 'webp']
limit = 50

# unused
def preprocess_text(text):
    # Remove non-printable characters and Unicode escape sequences
    text = text.encode('ascii', 'ignore').decode('utf-8')
    # TODO: Add preprocess steps as per data, Convert the text to lowercase
    text = text.lower()
    # # Tokenize the text into individual words
    tokens = word_tokenize(text)
    # # Remove stopwords and punctuation from the tokens
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    tokens = [token for token in tokens if token not in stop_words and token not in punctuation]
    return " ".join(tokens)
    
def dfs_web_scraping(root_url, tokenize = False, text_tag = 'article'):
    counter = 0
    while len(nodes_list)>0 and (counter < limit or limit == -1):
        counter = counter + 1
        node=nodes_list.pop()
        visited.add(node)
        fullnode = root_url + node
        print('Getting content from', fullnode)
        r = requests.get(fullnode, headers=HEADERS)
        soup = BeautifulSoup(r.content)
        title = soup.find('h1')
        paragraphs = soup.find(text_tag).findAll('p')
        article_text = ' '.join([p.text for p in paragraphs])
        
        # update nodes_list
        for a in soup.findAll('a'):
            # make sure the href exists and is internal
            if(not a.has_attr('href') or not a['href'].startswith('/')):
                continue
            # exclude files
            url_path = urlparse(a['href']).path
            file_extension = os.path.splitext(url_path)[1]
            if(file_extension in files_extension_to_exclude):
                continue
            # maintain stack
            if(a['href'] not in visited and a['href'] not in nodes_list):
                nodes_list.append(a['href'])
        
dfs_web_scraping('https://www.niaid.nih.gov')

Getting content from https://www.niaid.nih.gov/
Getting content from https://www.niaid.nih.gov/global/email-updates
Getting content from https://www.niaid.nih.gov/node/6476
Getting content from https://www.niaid.nih.gov/clinical-trials/participating-niaid-hiv-clinical-trial-spanish
Getting content from https://www.niaid.nih.gov/node/9994
Getting content from https://www.niaid.nih.gov/node/10480
Getting content from https://www.niaid.nih.gov/node/10525
Getting content from https://www.niaid.nih.gov/clinical-trials/pact-priority-protocols
Getting content from https://www.niaid.nih.gov/clinical-trials/whats-it-participate-pact-clinical-study
Getting content from https://www.niaid.nih.gov/clinical-trials/niaid-pact-patient-and-caregiver-resources
Getting content from https://www.niaid.nih.gov/node/2033
Getting content from https://www.niaid.nih.gov/node/12739
Getting content from https://www.niaid.nih.gov/node/2032
Getting content from https://www.niaid.nih.gov/clinical-trials/meet-pact-te

KeyboardInterrupt: 

In [ ]:
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "title": {"type": "string"},
        "description": {"type": "string"},
        "url": {"type": "string"}
    },
    "required": ["title", "description", "url"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

In [3]:
import pprint

from langchain_text_splitters import RecursiveCharacterTextSplitter


def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["span"]
    )
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content

In [4]:
urls = ["https://www.wsj.com"]
extracted_content = scrape_with_playwright(urls, schema=schema)

NameError: name 'schema' is not defined

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

docs[0]
splits = text_splitter.split_documents(docs)

NameError: name 'text_splitter' is not defined

In [8]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        print(type(history))
        return "", history + [[user_message, None]]

    def bot(history):
        print(type(history))
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=True).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


<class 'list'>
<class 'list'>
